In [1]:
import pandas as pd
import praw
import re
import requests

In [2]:
reddit = praw.Reddit(
    client_id="YOUR CLIENT ID",
    client_secret="YOUR CLIENT SECRET",
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
)


In [3]:
df = []

for post in reddit.subreddit('wallstreetbets').hot(limit=None):

    content = {
        "title": post.title,
        "text": post.selftext
    }

    df.append(content)

df = pd.DataFrame(df)

df

,title,text
0,Weekend Discussion Thread for the Weekend of J...,Your weekend discussion thread. Please keep th...
1,"Daily Popular Tickers Thread for June 11, 2021...",\nYour daily hype thread. Please keep the shit...
2,chartexchange now shows GME candles as crayons,
3,What it means to be a WSB member,
4,A Quick PSA,
...,...,...
383,Short interest fees rise in conjunction to big...,
384,Let’s go apes! $wish,
385,TLYS offering special $1 dividends per share f...,The clothing brand tilly's has had an explosiv...
386,"WISH : do not forget, Before or after the rall...",As Always I never give financial advice and th...


In [4]:
# Step 2: Analyze word frequency

regex = re.compile('[^a-zA-Z ]')
word_dict = {}

for (index, row) in df.iterrows():
    # titles
    title = row['title']

    title = regex.sub('', title)
    title_words = title.split(' ')

    # content
    content = row['text']

    content = regex.sub('', content)
    content_words = content.split(' ')

    # combine
    words = title_words + content_words

    for x in words:

        if x in ['A', 'B', 'GO', 'ARE', 'ON', 'IT', 'ALL', 'NEXT', 'PUMP', 'AT', 'NOW', 'FOR', 'TD', 'CEO', 'AM', 'K', 'BIG', 'BY', 'LOVE', 'CAN', 'BE', 'SO', 'OUT', 'STAY', 'OR', 'NEW', 'RH', 'EDIT', 'ONE', 'ANY']:
            pass
        elif x in word_dict:
            word_dict[x] += 1
        else:
            word_dict[x] = 1

word_df = pd.DataFrame.from_dict(list(word_dict.items())).rename(
    columns={0: "Term", 1: "Frequency"})

In [5]:
# Step 3: Get a list of stock tickers

ticker_df = pd.read_csv('tickers.csv').rename(
    columns={"Symbol": "Term", "Name": "Company_Name"})

# Step 4: Compare tickers and words scraped

stonks_df = pd.merge(ticker_df, word_df, on="Term")
stonks_df = stonks_df.sort_values(
    by="Frequency", ascending=False, ignore_index=True)
stonks_df.head(20)

,Term,Company_Name,Frequency
0,AMC,AMC Entertainment Holdings Inc. Class A Common...,83
1,CLF,Cleveland-Cliffs Inc. Common Stock,73
2,WISH,ContextLogic Inc. Class A Common Stock,68
3,GME,GameStop Corporation Common Stock,59
4,DD,DuPont de Nemours Inc. Common Stock,59
5,CLOV,Clover Health Investments Corp. Class A Common...,58
6,M,Macy's Inc Common Stock,52
7,CLNE,Clean Energy Fuels Corp. Common Stock,49
8,UWMC,UWM Holdings Corporation Class A Common Stock,49
9,BB,BlackBerry Limited Common Stock,43
